In [165]:
import pandas as pd
import numpy as np
from dateutil.relativedelta import *
from pandas import DataFrame
import datetime


In [166]:
## Read in match dataset
df = pd.read_csv(r'C:\Users\anoble\Git\Outside_Work\AFL Fantasy\Data\Inputs\fryziggafl.csv')
df['year'] = pd.DatetimeIndex(df['match_date']).year

## Filter to only games from 2010 onwards. Remove shortened season stats (2019 preds and 2020 stats)
df = df.query('year >= 2010').query('year != 2019').query('year != 2020')

## Read in Player Database
players = pd.read_csv(r'C:\Users\anoble\Git\Outside_Work\AFL Fantasy\Data\Inputs\player_ages.csv')

## Merge game stats with player info
df['player_name'] = df['player_last_name'].astype(str) + ", " + df['player_first_name']
joined_data = df.merge(players, on=['player_name'],how='left')

## Calculate age on match day (take year of game less year of birth, so will remain same age all season)
joined_data['age'] = pd.DatetimeIndex(joined_data['match_date']).year - pd.DatetimeIndex(joined_data['dob']).year

# Write to csv to check
# joined_data.to_csv(r'C:\Users\anoble\Git\Outside_Work\AFL Fantasy\Data\Inputs\fryziggafl_limited.csv')

# Aggregate 
avg_scores = joined_data.groupby(['player_team', 'player_first_name', 'player_last_name','player_name', 'age', 'year'])\
.agg({'afl_fantasy_score': 'mean'})\
.sort_values(by=['afl_fantasy_score'], ascending=False)\

pd.set_option('display.max_rows', 10)
# print(avg_scores)
avg_scores.query('age < 22').head(10)

C:\Users\anoble\AppData\Local\Temp\ipykernel_27952\989152385.py:2: DtypeWarning: Columns (25,80) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'C:\Users\anoble\Git\Outside_Work\AFL Fantasy\Data\Inputs\fryziggafl.csv')


afl_fantasy_score
player_team            player_first_name player_last_name player_name        age  year                   
Essendon               Zach              Merrett          Merrett, Zach      21.0 2016         117.590909
Sydney                 Errol             Gulden           Gulden, Errol      21.0 2023         112.250000
Gold Coast             Brayden           Fiorini          Fiorini, Brayden   19.0 2016         111.500000
Carlton                Sam               Walsh            Walsh, Sam         21.0 2021         108.954545
Collingwood            Nick              Daicos           Daicos, Nick       20.0 2023         108.409091
Melbourne              Clayton           Oliver           Oliver, Clayton    21.0 2018         107.160000
Western Bulldogs       Jack              Macrae           Macrae, Jack       20.0 2014         104.380952
                                                                             21.0 2015         103.857143
Collingwood            Steele            Sidebottom       Sidebottom, Steele 21.0 2012         103.541667
Greater Western Sydney Adam              Treloar          Treloar, Adam      21.0 2014         103.450000

# Data cleaning (Priyanshu). 
Regular season games only (exclude finals)
Exclude TOG <50%
Scale up 2020 numbers by 1.2x



In [167]:
# Identify the seasons played for each player. 
# Remove seasons where game played <=5 
# Rank seasons for each player - e.g. most recent season with >5 games = 1, then 2 and so on. 
# It will be different depending on player, for example 2021 for Zac Williams will be Season 2 whereas for Jack Steele thats season 3

# Games player per year. Filter to only where games >= 5, and remove 2023 as we have no response variable for recent scores
player_summ = joined_data.groupby(['player_name','year']).agg(matches=('match_date', 'count'),year_played=('year', 'mean'))\
    .query('matches >=5' )\

player_summ

matches  year_played
player_name       year                      
Aarts, Jake       2021       21       2021.0
                  2022        7       2022.0
Ablett, Gary      2010       24       2010.0
                  2011       20       2011.0
                  2012       20       2012.0
...                         ...          ...
van Berlo, Nathan 2015       20       2015.0
van Rooyen, Jacob 2023       20       2023.0
vandenBerg, Aaron 2015       14       2015.0
                  2016       14       2016.0
                  2018        7       2018.0

[6564 rows x 2 columns]

# Seasons active factor
 

In [168]:
seasons_active = player_summ.groupby(['player_name']).agg(years_active=('year_played', 'count'))

seasons_active

,years_active
player_name,
"Aarts, Jake",2
"Ablett, Gary",9
"Acres, Blake",7
"Adams, Leigh",5
"Adams, Marcus",5
...,...
"de Boer, Matt",10
"van Berlo, Jay",2
"van Berlo, Nathan",5


## In Season change factors

In [169]:
# ## Add scores during 1st half of season. Convert match round to numeric, return AFL score if valid, else return Null
joined_data['h1_fantasy_score'] = np.where(pd.to_numeric(joined_data['match_round'], errors='coerce') <= 10, joined_data['afl_fantasy_score'], np.NaN)
joined_data['h2_fantasy_score'] = np.where(pd.to_numeric(joined_data['match_round'], errors='coerce') > 10, joined_data['afl_fantasy_score'], np.NaN)


# Test
joined_data.agg({'afl_fantasy_score': 'mean','h1_fantasy_score': 'mean','h2_fantasy_score': 'mean'})

afl_fantasy_score    69.913854
h1_fantasy_score     69.993452
h2_fantasy_score     69.969046
dtype: float64

## In game stat factors

In [170]:
#Explode player data by years so that each year matches to every other active year
player_summ_exp = player_summ.merge(joined_data, on =['player_name'], how='inner')\
                

## Limit to games in the same year for 1yr factors, as well as 3 years up to the predicted season, and 5 years up to predicted season
player_summ_exp_1yr = player_summ_exp.query(('year_played == year'))
player_summ_exp_3yr = player_summ_exp.query(('year_played < year + 3')).query(('year_played >= year'))
player_summ_exp_5yr = player_summ_exp.query(('year_played < year + 5')).query(('year_played >= year'))

print(player_summ_exp.shape)
print(player_summ_exp_1yr.shape)
print(player_summ_exp_3yr.shape)
print(player_summ_exp_5yr.shape)

## Test matching works - should see 
pd.set_option('display.max_rows', 50)
player_summ_exp_3yr.groupby(['year_played','year']).agg(countall=('player_name', 'count')).head(50)

(728863, 92)
(107461, 92)
(239416, 92)
(318909, 92)


countall
year_played year          
2010.0      2010      8055
2011.0      2010      7088
            2011      8467
2012.0      2010      6357
            2011      7377
            2012      8940
2013.0      2011      6844
            2012      7852
            2013      8955
2014.0      2012      6994
            2013      7770
            2014      8950
2015.0      2013      6879
            2014      7834
            2015      8919
2016.0      2014      6672
            2015      7433
            2016      8913
2017.0      2015      6920
            2016      7714
            2017      8944
2018.0      2016      6755
            2017      7658
            2018      8926
2021.0      2021      9366
2022.0      2021      8147
            2022      9322
2023.0      2021      7450
            2022      8211
            2023      9704

In [171]:
## Average factors over 1 year of game data
model_factors_agg_1yr = player_summ_exp_1yr.groupby(['player_name', 'year_played'])\
.agg({'player_height_cm': 'mean',
'player_weight_kg': 'mean',
'age': 'mean',
'kicks': 'mean',
'marks': 'mean',
'handballs': 'mean',
'handballs': 'mean',
'disposals': 'mean',
'effective_disposals': 'mean',
'disposal_efficiency_percentage': 'mean',
'goals': 'mean',
'behinds': 'mean',
'hitouts': 'mean',
'tackles': 'mean',
'rebounds': 'mean',
'inside_fifties': 'mean',
'clearances': 'mean',
'clangers': 'mean',
'free_kicks_for': 'mean',
'free_kicks_against': 'mean',
'brownlow_votes': 'mean',
'contested_possessions': 'mean',
'uncontested_possessions': 'mean',
'contested_marks': 'mean',
'marks_inside_fifty': 'mean',
'one_percenters': 'mean',
'bounces': 'mean',
'goal_assists': 'mean',
'time_on_ground_percentage': 'mean',
'centre_clearances': 'mean',
'stoppage_clearances': 'mean',
'score_involvements': 'mean',
'metres_gained': 'mean',
'turnovers': 'mean',
'intercepts': 'mean',
'tackles_inside_fifty': 'mean',
'contest_def_losses': 'mean',
'contest_def_one_on_ones': 'mean',
'contest_off_one_on_ones': 'mean',
'contest_off_wins': 'mean',
'def_half_pressure_acts': 'mean',
'effective_kicks': 'mean',
'f50_ground_ball_gets': 'mean',
'ground_ball_gets': 'mean',
'hitouts_to_advantage': 'mean',
'hitout_win_percentage': 'mean',
'intercept_marks': 'mean',
'marks_on_lead': 'mean',
'pressure_acts': 'mean',
'rating_points': 'mean',
'ruck_contests': 'mean',
'score_launches': 'mean',
'shots_at_goal': 'mean',
'spoils': 'mean',
'afl_fantasy_score': 'mean',
'h1_fantasy_score': 'mean',
'h2_fantasy_score': 'mean'
})

model_factors_agg_1yr.columns = [str(col) + '_1yr' for col in model_factors_agg_1yr.columns]

model_factors_agg_1yr

player_height_cm_1yr  player_weight_kg_1yr  \
player_name       year_played                                               
Aarts, Jake       2021.0                      177.0                  76.0   
                  2022.0                      177.0                  76.0   
Ablett, Gary      2010.0                      182.0                  87.0   
                  2011.0                      182.0                  87.0   
                  2012.0                      182.0                  87.0   
...                                             ...                   ...   
van Berlo, Nathan 2015.0                        NaN                   NaN   
van Rooyen, Jacob 2023.0                      193.0                  96.0   
vandenBerg, Aaron 2015.0                      188.0                  94.0   
                  2016.0                      188.0                  94.0   
                  2018.0                      188.0                  94.0   

                               age_1yr  kicks_1yr  marks_1yr  handballs_1yr  \
player_name       year_played                                                 
Aarts, Jake       2021.0           NaN   5.809524   2.523810       4.761905   
                  2022.0           NaN   2.857143   0.857143       4.142857   
Ablett, Gary      2010.0           NaN  14.083333   4.416667      17.416667   
                  2011.0           NaN  15.200000   2.250000      15.050000   
                  2012.0           NaN  19.450000   3.050000      14.300000   
...                                ...        ...        ...            ...   
van Berlo, Nathan 2015.0           NaN   9.350000   3.950000       5.950000   
van Rooyen, Jacob 2023.0           NaN   4.500000   3.100000       4.200000   
vandenBerg, Aaron 2015.0           NaN  10.214286   3.142857       6.571429   
                  2016.0           NaN   5.714286   3.071429       8.928571   
                  2018.0           NaN   8.714286   4.285714       7.142857   

                               disposals_1yr  effective_disposals_1yr  \
player_name       year_played                                           
Aarts, Jake       2021.0           10.571429                 8.000000   
                  2022.0            7.000000                 4.857143   
Ablett, Gary      2010.0           31.500000                      NaN   
                  2011.0           30.250000                      NaN   
                  2012.0           33.750000                23.789474   
...                                      ...                      ...   
van Berlo, Nathan 2015.0           15.300000                10.400000   
van Rooyen, Jacob 2023.0            8.700000                 6.800000   
vandenBerg, Aaron 2015.0           16.785714                 9.785714   
                  2016.0           14.642857                 9.785714   
                  2018.0           15.857143                10.857143   

                               disposal_efficiency_percentage_1yr  goals_1yr  \
player_name       year_played                                                  
Aarts, Jake       2021.0                                69.809524   0.857143   
                  2022.0                                67.000000   0.285714   
Ablett, Gary      2010.0                                      NaN   1.833333   
                  2011.0                                      NaN   0.900000   
                  2012.0                                69.000000   1.300000   
...                                                           ...        ...   
van Berlo, Nathan 2015.0                                68.350000   0.200000   
van Rooyen, Jacob 2023.0                                78.800000   1.400000   
vandenBerg, Aaron 2015.0                                56.000000   0.357143   
                  2016.0                                66.785714   0.857143   
                  2018.0                                69.571429   0.714286   

          

In [172]:
# Build 3 Year Player Metrics
model_factors_agg_3yr = player_summ_exp_3yr.groupby(['player_name', 'year_played'])\
.agg({'kicks': 'mean',
'marks': 'mean',
'handballs': 'mean',
'handballs': 'mean',
'disposals': 'mean',
'effective_disposals': 'mean',
'disposal_efficiency_percentage': 'mean',
'goals': 'mean',
'behinds': 'mean',
'hitouts': 'mean',
'tackles': 'mean',
'rebounds': 'mean',
'inside_fifties': 'mean',
'clearances': 'mean',
'clangers': 'mean',
'free_kicks_for': 'mean',
'free_kicks_against': 'mean',
'brownlow_votes': 'mean',
'contested_possessions': 'mean',
'uncontested_possessions': 'mean',
'contested_marks': 'mean',
'marks_inside_fifty': 'mean',
'one_percenters': 'mean',
'bounces': 'mean',
'goal_assists': 'mean',
'time_on_ground_percentage': 'mean',
'centre_clearances': 'mean',
'stoppage_clearances': 'mean',
'score_involvements': 'mean',
'metres_gained': 'mean',
'turnovers': 'mean',
'intercepts': 'mean',
'tackles_inside_fifty': 'mean',
'contest_def_losses': 'mean',
'contest_def_one_on_ones': 'mean',
'contest_off_one_on_ones': 'mean',
'contest_off_wins': 'mean',
'def_half_pressure_acts': 'mean',
'effective_kicks': 'mean',
'f50_ground_ball_gets': 'mean',
'ground_ball_gets': 'mean',
'hitouts_to_advantage': 'mean',
'hitout_win_percentage': 'mean',
'intercept_marks': 'mean',
'marks_on_lead': 'mean',
'pressure_acts': 'mean',
'rating_points': 'mean',
'ruck_contests': 'mean',
'score_launches': 'mean',
'shots_at_goal': 'mean',
'spoils': 'mean',
'afl_fantasy_score': 'mean',
'h1_fantasy_score': 'mean',
'h2_fantasy_score': 'mean'
})

model_factors_agg_3yr.columns = [str(col) + '_3yr' for col in model_factors_agg_3yr.columns]

model_factors_agg_3yr

kicks_3yr  marks_3yr  handballs_3yr  \
player_name       year_played                                        
Aarts, Jake       2021.0        5.809524   2.523810       4.761905   
                  2022.0        5.071429   2.107143       4.607143   
Ablett, Gary      2010.0       14.083333   4.416667      17.416667   
                  2011.0       14.590909   3.431818      16.340909   
                  2012.0       16.109375   3.312500      15.703125   
...                                  ...        ...            ...   
van Berlo, Nathan 2015.0       10.153846   3.794872       5.974359   
van Rooyen, Jacob 2023.0        4.500000   3.100000       4.200000   
vandenBerg, Aaron 2015.0       10.214286   3.142857       6.571429   
                  2016.0        7.964286   3.107143       7.750000   
                  2018.0        6.714286   3.476190       8.333333   

                               disposals_3yr  effective_disposals_3yr  \
player_name       year_played                                           
Aarts, Jake       2021.0           10.571429                 8.000000   
                  2022.0            9.678571                 7.214286   
Ablett, Gary      2010.0           31.500000                      NaN   
                  2011.0           30.931818                      NaN   
                  2012.0           31.812500                23.789474   
...                                      ...                      ...   
van Berlo, Nathan 2015.0           16.128205                10.461538   
van Rooyen, Jacob 2023.0            8.700000                 6.800000   
vandenBerg, Aaron 2015.0           16.785714                 9.785714   
                  2016.0           15.714286                 9.785714   
                  2018.0           15.047619                10.142857   

                               disposal_efficiency_percentage_3yr  goals_3yr  \
player_name       year_played                                                  
Aarts, Jake       2021.0                                69.809524   0.857143   
                  2022.0                                69.107143   0.714286   
Ablett, Gary      2010.0                                      NaN   1.833333   
                  2011.0                                      NaN   1.409091   
                  2012.0                                69.000000   1.375000   
...                                                           ...        ...   
van Berlo, Nathan 2015.0                                65.282051   0.307692   
van Rooyen, Jacob 2023.0                                78.800000   1.400000   
vandenBerg, Aaron 2015.0                                56.000000   0.357143   
                  2016.0                                61.392857   0.607143   
                  2018.0                                67.714286   0.809524   

                               behinds_3yr  hitouts_3yr  tackles_3yr  ...  \
player_name       year_played                                         ...   
Aarts, Jake       2021.0          0.571429     0.000000     2.523810  ...   
                  2022.0          0.500000     0.000000     2.392857  ...   
Ablett, Gary      2010.0          1.083333     0.000000     4.208333  ...   
                  2011.0          0.977273     0.000000     5.000000  ...   
                  2012.0          0.968750     0.000000     5.156250  ...   
...                                    ...          ...          ...  ...   
van Berlo, Nathan 2015.0          0.384615     0.076923     3.487179  ...   
van Rooyen, Jacob 2023.0          0.450000     3.700000     2.150000  ...   
vandenBerg, Aaron 2015.0          0.857143     0.071429     5.000000  ...   
                  2016.0          0.750000     0.071429     4.285714  ...   
                  2018.0          0.714286     0.047619     3.428571  ...   

                               marks_on_lead_3yr  pressure_acts_3yr  \
player_name       year_played                           

In [173]:
# Build 5 Year Player Metrics

model_factors_agg_5yr = player_summ_exp_5yr.groupby(['player_name', 'year_played'])\
.agg({'kicks': 'mean',
'marks': 'mean',
'handballs': 'mean',
'handballs': 'mean',
'disposals': 'mean',
'effective_disposals': 'mean',
'disposal_efficiency_percentage': 'mean',
'goals': 'mean',
'behinds': 'mean',
'hitouts': 'mean',
'tackles': 'mean',
'rebounds': 'mean',
'inside_fifties': 'mean',
'clearances': 'mean',
'clangers': 'mean',
'free_kicks_for': 'mean',
'free_kicks_against': 'mean',
'brownlow_votes': 'mean',
'contested_possessions': 'mean',
'uncontested_possessions': 'mean',
'contested_marks': 'mean',
'marks_inside_fifty': 'mean',
'one_percenters': 'mean',
'bounces': 'mean',
'goal_assists': 'mean',
'time_on_ground_percentage': 'mean',
'centre_clearances': 'mean',
'stoppage_clearances': 'mean',
'score_involvements': 'mean',
'metres_gained': 'mean',
'turnovers': 'mean',
'intercepts': 'mean',
'tackles_inside_fifty': 'mean',
'contest_def_losses': 'mean',
'contest_def_one_on_ones': 'mean',
'contest_off_one_on_ones': 'mean',
'contest_off_wins': 'mean',
'def_half_pressure_acts': 'mean',
'effective_kicks': 'mean',
'f50_ground_ball_gets': 'mean',
'ground_ball_gets': 'mean',
'hitouts_to_advantage': 'mean',
'hitout_win_percentage': 'mean',
'intercept_marks': 'mean',
'marks_on_lead': 'mean',
'pressure_acts': 'mean',
'rating_points': 'mean',
'ruck_contests': 'mean',
'score_launches': 'mean',
'shots_at_goal': 'mean',
'spoils': 'mean',
'afl_fantasy_score': 'mean',
'h1_fantasy_score': 'mean',
'h2_fantasy_score': 'mean'
})

model_factors_agg_5yr.columns = [str(col) + '_5yr' for col in model_factors_agg_5yr.columns]

model_factors_agg_5yr

kicks_5yr  marks_5yr  handballs_5yr  \
player_name       year_played                                        
Aarts, Jake       2021.0        5.809524   2.523810       4.761905   
                  2022.0        5.071429   2.107143       4.607143   
Ablett, Gary      2010.0       14.083333   4.416667      17.416667   
                  2011.0       14.590909   3.431818      16.340909   
                  2012.0       16.109375   3.312500      15.703125   
...                                  ...        ...            ...   
van Berlo, Nathan 2015.0       11.290698   4.406977       7.337209   
van Rooyen, Jacob 2023.0        4.500000   3.100000       4.200000   
vandenBerg, Aaron 2015.0       10.214286   3.142857       6.571429   
                  2016.0        7.964286   3.107143       7.750000   
                  2018.0        8.114286   3.342857       7.628571   

                               disposals_5yr  effective_disposals_5yr  \
player_name       year_played                                           
Aarts, Jake       2021.0           10.571429                 8.000000   
                  2022.0            9.678571                 7.214286   
Ablett, Gary      2010.0           31.500000                      NaN   
                  2011.0           30.931818                      NaN   
                  2012.0           31.812500                23.789474   
...                                      ...                      ...   
van Berlo, Nathan 2015.0           18.627907                11.063492   
van Rooyen, Jacob 2023.0            8.700000                 6.800000   
vandenBerg, Aaron 2015.0           16.785714                 9.785714   
                  2016.0           15.714286                 9.785714   
                  2018.0           15.742857                10.000000   

                               disposal_efficiency_percentage_5yr  goals_5yr  \
player_name       year_played                                                  
Aarts, Jake       2021.0                                69.809524   0.857143   
                  2022.0                                69.107143   0.714286   
Ablett, Gary      2010.0                                      NaN   1.833333   
                  2011.0                                      NaN   1.409091   
                  2012.0                                69.000000   1.375000   
...                                                           ...        ...   
van Berlo, Nathan 2015.0                                66.687500   0.430233   
van Rooyen, Jacob 2023.0                                78.800000   1.400000   
vandenBerg, Aaron 2015.0                                56.000000   0.357143   
                  2016.0                                61.392857   0.607143   
                  2018.0                                63.028571   0.628571   

                               behinds_5yr  hitouts_5yr  tackles_5yr  ...  \
player_name       year_played                                         ...   
Aarts, Jake       2021.0          0.571429     0.000000     2.523810  ...   
                  2022.0          0.500000     0.000000     2.392857  ...   
Ablett, Gary      2010.0          1.083333     0.000000     4.208333  ...   
                  2011.0          0.977273     0.000000     5.000000  ...   
                  2012.0          0.968750     0.000000     5.156250  ...   
...                                    ...          ...          ...  ...   
van Berlo, Nathan 2015.0          0.441860     0.186047     3.930233  ...   
van Rooyen, Jacob 2023.0          0.450000     3.700000     2.150000  ...   
vandenBerg, Aaron 2015.0          0.857143     0.071429     5.000000  ...   
                  2016.0          0.750000     0.071429     4.285714  ...   
                  2018.0          0.771429     0.057143     4.057143  ...   

                               marks_on_lead_5yr  pressure_acts_5yr  \
player_name       year_played                           

In [174]:
# Most common position for the year
# Calculate most common position played over past year
# Consider grouping if needed to remove specificity (e.g. Left and Right indicators probably not useful)


# Games player per year. Filter to only where games >= 5
# position_mapp = joined_data.groupby(['player_name','year']).agg({'match_date': 'count'})\
#     .query('match_date >=5')

position_data = joined_data

position_data['position_grouped'] = np.select(
    [
     position_data['player_position'].isin(["FB"]), 
     position_data['player_position'].isin(["BPL", "BPR"]),
     position_data['player_position'].isin(["CHB"]), 
     position_data['player_position'].isin(["HBFL", "HBFR"]),
     position_data['player_position'].isin(["RK"]), 
     position_data['player_position'].isin(["C", "RR" ,"R"]),
     position_data['player_position'].isin(["WL", "WR"]), 
     position_data['player_position'].isin(["FF"]),
     position_data['player_position'].isin(["FPL", "FPR"]),
     position_data['player_position'].isin(["HFFL", "HFFR"]),
     position_data['player_position'].isin(["CHF"]),
     position_data['player_position'].isin(["INT", "SUB"])
    ],
    [
      'FB',
      'BP',
      'CHB',
      'HBF',
      'RUCK',
      'C',
      'W',
      'FF',
      'FP',
      'HFF',
      'CHF',
      'Ignore'  
    ], 
    default='Other'
     )

## Test no positions have not been allocated. Result = 0 rows
# joined_data.query('position_grouped in ["Other"]')\
#     .groupby(['player_position']).agg(matches=('match_date', 'count'))

position_data['matches'] = position_data.groupby(['player_name', 'year','position_grouped'])['match_id'].transform('count')

# Find most common position, ignoring sub and interchange scores
position_data['rank'] = position_data.query('position_grouped not in ["Ignore"]').groupby(['player_name', 'year'])['matches'].rank(method="first", ascending=False)

player_pos_yearly = position_data.query('rank == 1')[['player_name', 'year','position_grouped']]

# Test output
player_pos_yearly.query('player_name == "Himmelberg, Harry"')

,player_name,year,position_grouped
92293,"Himmelberg, Harry",2016,FP
92298,"Himmelberg, Harry",2017,FP
92308,"Himmelberg, Harry",2018,FF
92344,"Himmelberg, Harry",2021,CHF
92356,"Himmelberg, Harry",2022,CHF
92390,"Himmelberg, Harry",2023,CHB


## Response variable

In [175]:
# Average AFL fantasy score per season
response_var = joined_data.groupby(['player_name','year']).agg(matches=('match_date', 'count'),year_int=('year', 'mean'),avg_score=('afl_fantasy_score', 'mean'))\
    .query('matches >= 5').query('avg_score > 0')\
    .sort_values(by=['avg_score'], ascending=False)

# Subtract 1 year from AFL score so it matches to factors from last year
response_var['predicted_year'] = response_var['year_int'] - 1

response_var  

matches  year_int   avg_score  predicted_year
player_name    year                                               
Rockliff, Tom  2014       18    2014.0  134.777778          2013.0
Swan, Dane     2012       21    2012.0  130.666667          2011.0
Mitchell, Tom  2018       24    2018.0  128.166667          2017.0
               2017       22    2017.0  127.181818          2016.0
Ablett, Gary   2012       20    2012.0  124.700000          2011.0
...                      ...       ...         ...             ...
Wanganeen, Tex 2022        5    2022.0   17.600000          2021.0
Berry, Thomas  2021       11    2021.0   16.909091          2020.0
Morris, Josh   2021        5    2021.0    8.600000          2020.0
Davis, Ben     2022        5    2022.0    8.400000          2021.0
Chandler, Kade 2021        5    2021.0    2.200000          2020.0

[6564 rows x 4 columns]

## Join all factors together


In [176]:
## Player / year combos to include
factors = player_summ.merge(seasons_active, left_on=['player_name'], right_on=['player_name'], how='inner')\
           .merge(player_pos_yearly, left_on=['player_name', 'year_played'], right_on=['player_name', 'year'], how='inner')\
           .merge(model_factors_agg_1yr, left_on=['player_name', 'year_played'], right_on=['player_name', 'year_played'], how='inner')\
           .merge(model_factors_agg_3yr, left_on=['player_name', 'year_played'], right_on=['player_name', 'year_played'], how='inner')\
           .merge(model_factors_agg_5yr, left_on=['player_name', 'year_played'], right_on=['player_name', 'year_played'], how='inner')\
           .merge(response_var, left_on=['player_name', 'year_played'], right_on=['player_name', 'predicted_year'], how='left')\


## Add delta factors. (Lazy method for now, just using the 1yr 3yr and 5yr averages)

factors['fantasy_score_yearly_change'] = factors['afl_fantasy_score_1yr'] - factors['afl_fantasy_score_3yr']
factors['fantasy_score_inyear_change'] = factors['h2_fantasy_score_1yr'] - factors['h1_fantasy_score_1yr']



print(factors.shape)
print(factors.head(10))
factors.to_csv(r'C:\Users\anoble\Git\Outside_Work\AFL Fantasy\Data\Inputs\factors.csv',mode='w+')

(6434, 174)
    player_name  matches_x  year_played  years_active  year position_grouped  \
0   Aarts, Jake         21       2021.0             2  2021              HFF   
1   Aarts, Jake          7       2022.0             2  2022              HFF   
2  Ablett, Gary         24       2010.0             9  2010                C   
3  Ablett, Gary         20       2011.0             9  2011                C   
4  Ablett, Gary         20       2012.0             9  2012                C   
5  Ablett, Gary         21       2013.0             9  2013                C   
6  Ablett, Gary         15       2014.0             9  2014                C   
7  Ablett, Gary          6       2015.0             9  2015                C   
8  Ablett, Gary         14       2016.0             9  2016                C   
9  Ablett, Gary         14       2017.0             9  2017                C   

   player_height_cm_1yr  player_weight_kg_1yr  age_1yr  kicks_1yr  ...  \
0                 177.0          

In [179]:
factors[['afl_fantasy_score_1yr','afl_fantasy_score_3yr','fantasy_score_yearly_change', 'fantasy_score_inyear_change']]

,afl_fantasy_score_1yr,afl_fantasy_score_3yr,fantasy_score_yearly_change,fantasy_score_inyear_change
0,47.428571,47.428571,0.000000,-10.063636
1,27.571429,42.464286,-14.892857,16.700000
2,116.916667,116.916667,0.000000,-15.083333
3,112.150000,114.750000,-2.600000,17.153846
4,124.700000,117.859375,6.840625,-11.541667
...,...,...,...,...
6429,67.000000,67.717949,-0.717949,-6.850000
6430,50.900000,50.900000,0.000000,4.545455
6431,73.928571,73.928571,0.000000,-3.400000
6432,61.142857,67.535714,-6.392857,9.939394
